# Проектный практикум 3

Создать нейронную сеть, способную генерировать текстовые отзывы о различных местах на основе определенных входных параметров, таких как категория места, средний рейтинг и ключевые слова.

## 1. Загрузка данных

In [ ]:
!pip install torch

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import numpy as np
import gensim
import spacy
import torch
import os
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [ ]:
# Загружаем датасет
!wget https://github.com/yandex/geo-reviews-dataset-2023/raw/master/geo-reviews-dataset-2023.tskv

--2024-12-18 19:18:27--  https://github.com/yandex/geo-reviews-dataset-2023/raw/master/geo-reviews-dataset-2023.tskv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/yandex/geo-reviews-dataset-2023/master/geo-reviews-dataset-2023.tskv [following]
--2024-12-18 19:18:27--  https://media.githubusercontent.com/media/yandex/geo-reviews-dataset-2023/master/geo-reviews-dataset-2023.tskv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 378730064 (361M) [application/octet-stream]
Saving to: ‘geo-reviews-dataset-2023.tskv’

geo-reviews-dataset 100%[===================>] 361.18M   174MB/s    

In [ ]:
# Путь к вашему файлу .tskv
file_path = 'geo-reviews-dataset-2023.tskv'

In [ ]:
# Список для хранения данных
data = []

# Чтение файла построчно
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Удаляем пробелы и символы новой строки
        line = line.strip()
        if line:  # Проверяем, что строка не пустая
            # Разделяем строку на пары "ключ=значение"
            items = line.split('\t')  # tskv использует табуляцию как разделитель
            data_dict = {}
            for item in items:
                key, value = item.split('=', 1)  # Разделяем только по первому '='
                data_dict[key] = value
            data.append(data_dict)

# Создание DataFrame из списка словарей
df = pd.DataFrame(data)

# Вывод первых нескольких строк DataFrame
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


## 2. Исследовательский анализ данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  500000 non-null  object
 1   name_ru  499030 non-null  object
 2   rating   500000 non-null  object
 3   rubrics  500000 non-null  object
 4   text     500000 non-null  object
dtypes: object(5)
memory usage: 19.1+ MB


In [ ]:
# Анализ рубрик
df.rubrics.value_counts()

,count
rubrics,
Гостиница,42242
Ресторан,14615
Кафе,12366
Супермаркет,8899
Магазин продуктов,5289
...,...
Технические и медицинские газы;Сварочные работы;Кованые изделия,1
"Ресторан;Бар, паб;Кафе;Столовая",1
Буровые работы;Нефтегазовая компания,1


In [ ]:
# Разделение рубрик по разделителю и создание новых строк
df['rubrics'] = df['rubrics'].str.replace(';', ',')  # Сначала заменяем ';' на ','
df['rubrics'] = df['rubrics'].str.split(',')  # Теперь разбиваем по ','
df = df.explode('rubrics')

df.rubrics.value_counts()

,count
rubrics,
Кафе,58496
Ресторан,56761
Гостиница,43133
Магазин продуктов,21346
Супермаркет,19746
...,...
Чистка и ремонт колодцев,1
Производство кормов для домашних животных,1
Геральдика и генеалогия,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998606 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  998606 non-null  object
 1   name_ru  996495 non-null  object
 2   rating   998606 non-null  object
 3   rubrics  998606 non-null  object
 4   text     998606 non-null  object
dtypes: object(5)
memory usage: 45.7+ MB


In [ ]:
# Поиск дубликатов
duplicates = df.duplicated(subset=['text'])

num_duplicates = duplicates.sum()
print(f"Количество дубликатов: {num_duplicates}")

Количество дубликатов: 498744


In [ ]:
# Удаление дубликатов
df = df.drop_duplicates(subset=['text'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499862 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  499862 non-null  object
 1   name_ru  498895 non-null  object
 2   rating   499862 non-null  object
 3   rubrics  499862 non-null  object
 4   text     499862 non-null  object
dtypes: object(5)
memory usage: 22.9+ MB


In [ ]:
# Анализ распределения целевых классов
df.rating.value_counts()

,count
rating,
5.,390383
4.,41154
1.,34351
3.,21686
2.,12088
0.,200


In [ ]:
# Очистка рейтинга от точек и преобразование в целые числа
df['rating'] = df['rating'].str.replace('.', '', regex=False).astype(int)

In [ ]:
df.rating.value_counts()

,count
rating,
5,390383
4,41154
1,34351
3,21686
2,12088
0,200


Потребители, по всей видимости, склонны оставлять позитивные отзывы.

In [ ]:
number_unique_name = df.name_ru.nunique()
print('Количество уникальных наименований организаций: {}'.format(number_unique_name))

Количество уникальных наименований организаций: 148442


In [ ]:
number_unique_address = df.address.nunique()
print('Количество уникальных адресов организаций: {}'.format(number_unique_address))

Количество уникальных адресов организаций: 191869


Уникальных адресов на сорок две с лишним тысячи бальше чем уникальных ораганизаций. Это означает, что некоторые организации работают по нескольким адресам. По каждому адресу могут быть различные отзывы потребителей.

In [ ]:
number_unique_rubrics = df.rubrics.nunique()
print('Количество уникальных рубрик: {}'.format(number_unique_rubrics))

Количество уникальных рубрик: 1253


In [ ]:
number_unique_text = df.text.nunique()
print('Количество уникальных текстов: {}'.format(number_unique_text))

Количество уникальных текстов: 499862


In [ ]:
text_by_name = df.groupby('name_ru')['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_name

,count
name_ru,
Пятёрочка,6026
Магнит,2609
Красное&Белое,1731
Wildberries,1692
Ozon,1492
...,...
Лингва Плюс,1
VIP шашлык,1
Линарис,1


In [ ]:
text_by_address_name = df.groupby(['address', 'name_ru'])['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_address_name

count
address                                            name_ru                                                  
Москва, проспект Андропова, 1                      Остров мечты                                          226
Москва, Большая Грузинская улица, 1с1              Московский зоопарк                                    162
Краснодарский край, городской округ Сочи, аэроп... Международный аэропорт Сочи имени В. И. Севасть...    155
Краснодар, Городской сад                           Парк Краснодар                                        151
Москва, Голубинская улица, 16                      Мореон                                                143
...                                                                                                      ...
Москва, улица Кирпичные Выемки, 2, корп. 1         Бигарден                                                1
                                                   Власть взгляда                                          1
                                                   Элна-мебель                                             1
Москва, улица Кирпичные Выемки, 2к1                Ozon                                                    1
​М-5 1705 километр, 2а пос. Березовый Мост, Сат... Лесная жемчужина                                        1

[275195 rows x 1 columns]

In [ ]:
text_by_rubrics = df.groupby('rubrics')['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_rubrics.head(20)

,count
rubrics,
Гостиница,42664
Ресторан,39793
Кафе,31274
Супермаркет,14373
Салон красоты,11951
Магазин продуктов,11790
Быстрое питание,9632
Торговый центр,8071
Музей,8048


Отзывы потребители в основном оставляют в сферах гостиничных услуг, общественного питания, торговли, индустрии красоты.

## Выводы по разделу

- в среднем на одну организацию приходится чуть более 3 отзывов.
- рубрик очень много, что затрудняет анализ данных и может снизить качество генерации текста. При этом многое рубрики пересекаются. По всей видимости их лучше разметить вручную.

## План дальнейшей работы

- Для генерации текстов о местах оказания услуг и продажи товаров проверим следующий подход. Что интересует потребителя? Прежде всего товар или услуга (в нашем случае рубрика), место ее оказания, поставки и качество поставщика. Исходя из этого создадаим последовательные фильтры оптимизации поиска: фильтр по рубрике - фильтр по городу - фильтр по улице - фильтр по рейтингу. После фильтрации появится список организаций, по которым пользователь, с помощью нейронной сети получит характеристику организаций.
-В следующем разделе создадим фильтры для поиска поставщиков.
- Проведем предобработку текстов отзывов для применения модели суммаризации.
- Далее протестируем модели суммаризации текстов отзывов.

## 3. Предобработка текстов отзывов для тестирования моделей

In [ ]:
# Вывод строк с отзывами с индексами от 15 до 23
for i in range(10, 34):
    print(df.text.iloc[i])
    print('---')

Очень большой выбор обуви для всей семьи, по разным ценам)))) Мне магазин очень понравился. Плюс всегда действует акция 2+1.
---
Очень сложно добраться на пляж по сломанной лестнице, рядом порт, с которого постоянной слышен грохот и идет гарь с работающих двигателей кораблей. В воде много стекол и острых предметов. Песок грязный и его практически нет - одни камни. На этот пляж ходить не советую, лучше поехать на Центральный пляж или платный Елисеевский, рядом с аквапарком Лазурный.
---
Вкусное место в центре города.  Разнообразное меню.  Отзывчивый персонал.  Несколько залов разной вместимости дают возможность проводить крупные мероприятия. Вкусная кондитерская на первом этаже.
---
Самый большой плюс это месторасположение, набережная , шикарный вид на море! Красиво, уютно, вот собственно плюсы закончились .. огорчает отношение к посетителям, официанты неприветливые, не здравствуйте вам, не до свидания . Лица недовольные, неприятные, больше не хочется смотреть на такие! Кухня тоже остав

### Уберем из текста эмодзи, лишние пробелы. Сохраним знаки препинания, заглавные буквы, так как это может способствовать повышению качеству модели суммаризации текста

In [ ]:
# Очистка текста
def clear_text(text):
    # Удаление хэштегов
    text = re.sub(r"#(\w+)", r"\1", text)

    # Удаление эмодзи, сохраняя цифры, знаки препинания и знаки плюс и минус
    text = re.sub(r"[^\w\s,.!?;:0-9+-]", "", text)  # Сохраняем + и - для акций и диапазонов

    # Удаляем все символы n и любые другие нежелательные символы
    text = re.sub(r'n', '', text)

    # Удаление лишних пробелов
    text = " ".join(text.split())

    return text


# Очистка и лемматизация
df['clear_text'] = df['text'].apply(clear_text)

In [ ]:
# Вывод очищенных строк с отзывами с индексами от 15 до 23
for i in range(10, 34):
    print(df.clear_text.iloc[i])
    print('---')

Очень большой выбор обуви для всей семьи, по разным ценам Мне магазин очень понравился. Плюс всегда действует акция 2+1.
---
Очень сложно добраться на пляж по сломанной лестнице, рядом порт, с которого постоянной слышен грохот и идет гарь с работающих двигателей кораблей. В воде много стекол и острых предметов. Песок грязный и его практически нет - одни камни. На этот пляж ходить не советую, лучше поехать на Центральный пляж или платный Елисеевский, рядом с аквапарком Лазурный.
---
Вкусное место в центре города. Разнообразное меню. Отзывчивый персонал. Несколько залов разной вместимости дают возможность проводить крупные мероприятия. Вкусная кондитерская на первом этаже.
---
Самый большой плюс это месторасположение, набережная , шикарный вид на море! Красиво, уютно, вот собственно плюсы закончились .. огорчает отношение к посетителям, официанты неприветливые, не здравствуйте вам, не до свидания . Лица недовольные, неприятные, больше не хочется смотреть на такие! Кухня тоже оставляет же

## 4. Обучение модели суммаризации текста

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
# Функция токенизации
def tokenize_by_spacy(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return tokens

In [ ]:
# Определяем параметры модели для выбора лучших
vector_sizes = [100, 150, 200]
windows = [5, 7, 10]
min_counts = [1, 2]
workers = [1]  # Обычно одно значение
sg_values = [0, 1]  # CBOW или Skip-gram

### Характеристика параметров модели:

- sentences: это входные данные для обучения, представляющие собой список списков токенов (слов). Каждый внутренний список соответствует одному предложению.
- vector_size: Размерность векторов, представляющих слова. Например, vector_size=100 означает, что каждое слово будет представлено вектором размерности 100. Увеличение этого значения может помочь захватить больше информации о семантике слов, поэтому попробуем несколько значения этого параметра.
- window: Максимальное расстояние между текущим словом и словами вокруг него. Например, window=5 означает, что модель будет учитывать до 5 слов слева и 5 слов справа от текущего слова. Увеличение этого значения может помочь модели лучше понимать контекст. Также попробуем обучить модель на разных значениях этого параметра.
- min_count: Минимальное количество раз, которое слово должно встречаться в корпусе, чтобы оно было включено в модель. Например, min_count=1 включает все слова. Увеличение этого значения может помочь избавиться от редких слов и уменьшить размер словаря.
- workers: Количество потоков для параллельной обработки данных. Значение по умолчанию — 1. Поскольку по каждой организации не много отзывов оставим этот параметр по умолчанию.
- sg: Определяет алгоритм обучения: если sg=0, используется CBOW (Continuous Bag of Words), если sg=1, используется Skip-Gram. CBOW быстрее обучается на больших объемах данных, но Skip-Gram лучше справляется с редкими словами.

In [ ]:
# Функция обучения модели Word2Vec
def train_word2vec_model(sentences, params):
    model = gensim.models.Word2Vec(sentences,
                                     vector_size=params['vector_size'],
                                     window=params['window'],
                                     min_count=params['min_count'],
                                     workers=params['workers'],
                                     sg=params['sg'])
    model.train(sentences, total_examples=model.corpus_count, epochs=1)
    return model

In [ ]:
# Функция получения векторного представления предложения
def get_sentence_vector(sentence, model):
    vector = np.zeros(model.vector_size)
    count = 0

    for word in sentence:
        if word in model.wv:
            vector += model.wv[word]
            count += 1

    if count > 0:
        vector /= count
    return vector

In [ ]:
# Функция для извлечения резюме
def extractive_summary(text, model, top_n=3):
    doc = nlp(text)  # Используем spaCy для обработки текста
    sentences = [sent.text for sent in doc.sents]  # Разбиение текста на предложения с помощью spaCy
    tokenized_sentences = [tokenize_by_spacy(sentence) for sentence in sentences]

    # Получение векторов для всех предложений
    sentence_vectors = np.array([get_sentence_vector(sentence,
                                                     model) for sentence in tokenized_sentences])

    # Вычисление матрицы схожести между предложениями
    similarity_matrix = cosine_similarity(sentence_vectors)

    # Ранжирование предложений по значимости
    scores = similarity_matrix.sum(axis=1)

    # Получение индексов наиболее значимых предложений
    ranked_indices = np.argsort(scores)[-top_n:][::-1]

    # Формирование итогового резюме
    summary = [sentences[i] for i in ranked_indices]

    return summary

In [ ]:
# Ввод названия города пользователем
city_name = input("Введите название города: ").strip()

# Поиск совпадений по городу в колонке address
matching_rows = df[df['address'].str.contains(city_name, case=False)]

# Проверка на наличие найденных совпадений
if not matching_rows.empty:
    # Ввод названия улицы пользователем
    street_name = input("Введите название улицы: ").strip()

    # Фильтрация по выбранной улице
    street_filtered = matching_rows[matching_rows['address'].str.contains(street_name, case=False)]

    # Проверка на наличие найденных совпадений по улице
    if not street_filtered.empty:
        # Ввод рубрики пользователем
        rubric_name = input("Введите название рубрики: ").strip()

        # Фильтрация по выбранной рубрике
        rubric_filtered = street_filtered[street_filtered['rubrics'].str.contains(rubric_name, case=False)]

        # Проверка на наличие найденных совпадений по рубрике
        if not rubric_filtered.empty:
            # Ввод минимального рейтинга пользователем
            min_rating = float(input("Введите минимальный рейтинг (например, 4.0): "))

            # Фильтрация по рейтингу
            final_results = rubric_filtered[rubric_filtered['rating'] >= min_rating]

            # Проверка на наличие найденных совпадений по рейтингу
            if not final_results.empty:
                result_names = final_results['name_ru'].unique()
                print("Найденные организации:")
                print(result_names)

                # Словарь для хранения всех суммаризаций по параметрам
                all_summaries = {}

                # Перебор всех комбинаций гиперпараметров
                for vector_size in vector_sizes:
                    for window in windows:
                        for min_count in min_counts:
                            for sg in sg_values:
                                # Определяем параметры модели
                                model_params = {
                                    'vector_size': vector_size,
                                    'window': window,
                                    'min_count': min_count,
                                    'workers': 1,
                                    'sg': sg
                                }

                                # Токенизация отзывов перед обучением модели Word2Vec
                                tokenized_reviews = [tokenize_by_spacy(review) for review in final_results['clear_text'].tolist()]

                                # Обучаем модель на отзывах для каждой организации
                                word2vec_model = train_word2vec_model(tokenized_reviews, model_params)

                                # Получаем резюме по всем отзывам для каждой организации
                                summaries = {}
                                for name in result_names:
                                    supplier_reviews = final_results[final_results['name_ru'] == name]['clear_text'].tolist()
                                    combined_reviews_text = ' '.join(supplier_reviews)

                                    summary = extractive_summary(combined_reviews_text, word2vec_model)
                                    summaries[name] = summary

                                # Сохраняем результаты для текущих параметров
                                all_summaries[(vector_size, window, min_count, sg)] = summaries

                # Вывод результатов по каждой комбинации гиперпараметров
                for params, summaries in all_summaries.items():
                    print(f"\nПараметры: vector_size={params[0]}, window={params[1]}, min_count={params[2]}, sg={params[3]}")
                    for supplier, summary in summaries.items():
                        print(f"{supplier}: {' '.join(summary)}")
                    print("\n" + "="*50 + "\n")  # Разделитель между результатами

            else:
                print(f"В городе '{city_name}' на улице '{street_name}' нет организаций в рубрике '{rubric_name}' с рейтингом не ниже {min_rating}.")
        else:
            print(f"В городе '{city_name}' на улице '{street_name}' нет организаций в рубрике '{rubric_name}'.")
    else:
        print(f"В городе '{city_name}' нет организаций на улице '{street_name}'.")
else:
    print(f"Город '{city_name}' не найден в адресах.")

Введите название города: калуга
Введите название улицы: ленина
Введите название рубрики: ресторан
Введите минимальный рейтинг (например, 4.0): 4
Найденные организации:
['Мир морей' 'Pro Italia']



Параметры: vector_size=100, window=5, min_count=1, sg=0
Мир морей: Отличное меню, все блюда интересны, хочется попробовать всё. Понравился интерьер, ненавязчиво и стильно. Всё было очень вкусно.
Pro Italia: Персонал вежливый! Очень приятное заведение! Посидеть помечтать!



Параметры: vector_size=100, window=5, min_count=1, sg=1
Мир морей: Отличное меню, все блюда интересны, хочется попробовать всё. Понравился интерьер, ненавязчиво и стильно. Роллы, сервисе, крудо, осьминоги, устрицы, ежи.
Pro Italia: Персонал вежливый! Очень приятное заведение! Очень вкусный манговый чай и панакота!



Параметры: vector_size=100, window=5, min_count=2, sg=0
Мир морей: Главное помнить, что в этом ресторане дорого. А спиртное можно принести с собой, оплатив пробковый сбор 300 рублей. Понравился интерьер, ненавязчиво и стильно.
Pro Italia: Очень вкусный манговый чай и панакота! Посидеть помечтать! Мини Италия!



Параметры: vector_size=100, window=5, min_count=2, sg=1
Мир морей: Главное помнить, что в э

### Лучший результат суммаризации показала модель с параметрами vector_size=200, window=10, min_count=1, sg=0

# Общие выводы по проекту

1. В проекте применена классическая экстрактивная модель суммаризации текстов отзывов о поставщиках.
2. При разных параметрах модель по разному суммаризует тексты, что говорит о ее работоспособности.
3. Вместе с тем применение современных генеративных моделей - GPT, Perplexity дали ба лучший результат суммаризации.